In [125]:
# Importación de librerias:
import redis
import hashlib
import requests
import json
import geopy.distance
import pymongo
from pymongo import MongoClient
import time
import airportsdata
import pandas as pd
import datetime

In [7]:
r = redis.Redis(host='127.0.0.1', port=6379, db=0)

In [49]:
client = MongoClient()
db = client.proyecto
vuelos = db.vuelos

In [15]:
# Generamos la función que especifique el estatus del vuelo.

def status(coords_2):
    coords_1 = (19.436111, -99.071944)
    dis =  geopy.distance.geodesic(coords_1, coords_2).km
    if dis < 20:
        return('L')
    else:
        return('A')

In [140]:
def ingresa(air_code, maxflights):
    
    airports = airportsdata.load('IATA')  # key is IATA code
    
    # Conexión a API
    url = 'https://api.flightstats.com/flex/flightstatus/rest/v2/json/airport/tracks/'
    appkey = 'fd2a2f47db6ed8f21988112c693c5d38'
    appid = '13f54a6b'
    maxflights = str(maxflights)
    url_req = url + air_code + '/arr?appId=' + appid + '&appKey=' + appkey + '%09&includeFlightPlan=false&maxPositions=1&maxFlights=' + maxflights
    
    response = requests.request("GET", url_req).json()
    
    for ubi in response['flightTracks']:
        # Extracción de datos
        id = str(ubi['flightId'])
        avion = ubi['callsign']
        lat = str(ubi['positions'][0]['lat'])
        long = str(ubi['positions'][0]['lon'])
        time = ubi['positions'][0]['date']
        velocidad = ubi['positions'][0]['speedMph']
        from_ = ubi['departureAirportFsCode']
        try:
            retraso = Diccionario['delayMinutes']
        except:
            retraso = 0
            
         # Ingreso a base de datos temporal
        if status((lat, long)) == 'A':
            r.set(id+':avion', avion)
            r.set(id+':lat', lat)
            r.set(id+':long', long)
            r.set(id+':time', time)
            r.set(id+':velocidad', velocidad)
            r.set(id+':from', from_)
            r.set(id+':retraso', retraso)
            
        if status((lat, long)) == 'L':
            r.delete(id+':avion', id+':lat', id+':long', id+':time', id+':velocidad', id+':from', id+':retraso')
            
            # Ingreso a base de datos histórica
            if len(list(vuelos.find({'_id':id}))) == 0:
                time = datetime.datetime.strptime(time[:-5], '%Y-%m-%dT%H:%M:%S')
                datos = {'_id': id, 'avion': avion, 'from': from_, 'retraso': retraso, 'time': time, 'ciudad':airports[from_]['city'],
                        'pais':airports[from_]['country']}
                vuelos.insert_one(datos)


In [141]:
# Iniciamos la simulación del flujo

def flujo(sleep, total):
    t = time.time()
    while True:
        ingresa('MEX', 100)
        time.sleep(sleep)
        if (time.time() - t) > total:
           break

In [142]:
flujo(30,60)

# Checar input de fecha en mongos

In [ ]:
# Consultas histórica:
# Promedio de delay de todo +
# Delay por aeropueto +
# Delay por pais +
# Delay por tiempo +
# De qué aeropuerto vienen más +
# Hora (tiempo) de llegada más común + 

# Conultas actuales
# Mapa tiempo real
# Aviones más rápidos actuales +

In [151]:
def promedio_retraso(tipo='Todo'):
    if tipo == 'Todo':
        return list(vuelos.aggregate([{'$group':{'_id':None, 'promedio':{'$avg':'$retraso'}}}]))[0]['promedio']
    if tipo == 'Aeropuerto':
        consulta = pd.DataFrame(list(vuelos.aggregate([{'$group':{'_id':'$from', 'promedio':{'$avg':'$retraso'}}}])))
        consulta.columns = ['Aeropuerto', 'Promedio']
        return consulta
    if tipo == 'Pais':
        consulta = pd.DataFrame(list(vuelos.aggregate([{'$group':{'_id':'$pais', 'promedio':{'$avg':'$retraso'}}}])))
        consulta.columns = ['Pais', 'Promedio']
        return consulta
    if tipo == 'Hora':  
        consulta = pd.DataFrame(list(vuelos.aggregate([{'$group':{'_id':{'$hour':'$time'}, 'promedio':{'$avg':'$retraso'}}}])))
        consulta.columns = ['Hora', 'Promedio']
        return consulta


In [157]:
def llegadas(tipo = 'Aeropuerto'):
    if tipo == 'Aeropuerto':
        consulta = pd.DataFrame(list(vuelos.aggregate([{'$group':{'_id':'$from', 'cuenta':{'$count':{}}}}])))
        consulta.columns = ['Aeropuerto', 'Cuenta']
        return consulta
    if tipo == 'Hora':
        consulta = pd.DataFrame(list(vuelos.aggregate([{'$group':{'_id':{'$hour':'$time'}, 'cuenta':{'$count':{}}}}])))
        consulta.columns = ['Aeropuerto', 'Cuenta']
        return consulta
    

In [161]:
def consulta_categoria(categoria):
    patron = '*' + categoria + '*'
    for key in r.scan_iter(patron):
        print(key)
        print(r.get(key))

In [208]:
def speed(limit = 10, rapidos=True):
    assert isinstance(rapidos, bool), 'Variable rapidos debe ser boolena'
    dicc = {}
    
    velo = '*velocidad*'
    for key in r.scan_iter(velo):
        dicc[key[:-10].decode("utf-8") ] = [r.get(key).decode("utf-8")]
        
    avion = '*avion*'
    for key in r.scan_iter(avion):
        dicc[key[:-6].decode("utf-8") ].append(r.get(key).decode("utf-8"))
        
    if rapidos:
        return sorted(list(dicc.values()), reverse=True)[:limit]
    else:
         return sorted(list(dicc.values()), reverse=False)[:limit]


In [217]:
def lat_long():
    dicc = {}
    
    lat = '*lat*'
    for key in r.scan_iter(lat):
        dicc[key[:-4].decode("utf-8") ] = [r.get(key).decode("utf-8")]
        
    long = '*long*'
    for key in r.scan_iter(long):
        dicc[key[:-5].decode("utf-8") ].append(r.get(key).decode("utf-8"))
        
    avion = '*avion*'
    for key in r.scan_iter(avion):
        dicc[key[:-6].decode("utf-8") ].append(r.get(key).decode("utf-8"))
        
    return dicc


In [218]:
lat_long()

{'1094308632': ['11.9623', '-87.2061', 'AMX691'],
 '1094308615': ['18.4047', '-97.7587', 'SLI675'],
 '1094308699': ['21.1653', '-90.5628', 'AMX831'],
 '1094374068': ['20.2954', '-97.2092', 'VOI713'],
 '1094359285': ['32.8652', '-116.0388', 'UAL1271'],
 '1094308330': ['19.2429', '-101.1785', 'AMX229'],
 '1094320786': ['23.1115', '-103.0149', 'DAL631'],
 '1094337737': ['17.0331', '-98.2366', 'LPE2472'],
 '1094308668': ['27.1599', '-99.6582', 'SLI633'],
 '1094373824': ['18.9689', '-99.314', 'VOI167'],
 '1094308618': ['32.6761', '-116.174', 'AMX645'],
 '1094373816': ['17.6694', '-99.7456', 'VOI107'],
 '1094373860': ['20.2027', '-103.152', 'VOI251'],
 '1094308290': ['21.787', '-102.1488', 'SLI165'],
 '1094364589': ['20.7101', '-90.8824', 'VIV1019'],
 '1094374222': ['19.6921', '-99.2506', 'VOI855'],
 '1094308308': ['23.2815', '-95.8574', 'AMX002'],
 '1094332187': ['37.0149', '-80.2362', 'IBE6403'],
 '1094373820': ['18.6268', '-98.0108', 'VOI137'],
 '1094308289': ['21.7806', '-101.679', 'SLI2

In [214]:
speed(2, False)

[['168', 'TNO303'], ['168', 'VOI223']]

In [158]:
llegadas('Hora')

,Aeropuerto,Cuenta
0,19,2


In [152]:
promedio_retraso('Hora')

,Hora,Promedio
0,19,0.0
